In [1]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [5]:
def rank(text):
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    filtered_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    scores=dict()
    for i in range(len(filtered_words)):
        for j in range(min(len(filtered_words),i+3)):
            scores.setdefault(filtered_words[i],0)
            scores.setdefault(filtered_words[j],0)
            similarity_score=model.similarity(filtered_words[i],filtered_words[j])
            scores[filtered_words[i]]+=similarity_score
            scores[filtered_words[j]]+=similarity_score
    return sorted(scores)

In [6]:
rank("Word embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with many dimensions per word to a continuous vector space with a much lower dimension.")

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # Remove the CWD from sys.path while we load stuff.


['(',
 ')',
 '.',
 'Conceptually',
 'NLP',
 'Word',
 'collective',
 'continuous',
 'dimension',
 'embedding',
 'feature',
 'involves',
 'language',
 'learning',
 'lower',
 'many',
 'mapped',
 'mathematical',
 'modeling',
 'much',
 'name',
 'natural',
 'number',
 'per',
 'phrase',
 'processing',
 'real',
 'set',
 'space',
 'technique',
 'vector',
 'vocabulary',
 'word']